In [14]:
import pandas as pd
import numpy as np
df_train_rides = pd.read_csv("../DBtrainrides_info_cleaned.csv")
df_train_rides.loc[df_train_rides['stop_number'] == 1, 'IBNR'] = df_train_rides['starting_station_IBNR']
df_train_rides

C:\Users\rapha\AppData\Local\Temp\ipykernel_15116\157855247.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_rides = pd.read_csv("../DBtrainrides_info_cleaned.csv")


,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,city,zip,last_station,IBNR,long,...,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,canceled,info_present,transformed_info_message
0,-2065137557584893414,2407082237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,NaN,2024-07-08 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message
1,-2065137557584893414,2407092237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,NaN,2024-07-09 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message
2,-2065137557584893414,2407102237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,NaN,2024-07-10 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message
3,-2065137557584893414,2407112237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,NaN,2024-07-11 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message
4,-2065137557584893414,2407122237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,NaN,2024-07-12 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811475,6234297817509604666,2407112012,7,70,8098360,Bürstadt,68642,stockstadt(rhein),8005740.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
2811476,6234297817509604666,2407112012,8,70,8098360,Bürstadt,68642,biebesheim,8000951.0,8.473978,...,2024-07-11 20:45:00,2024-07-11 20:45:00,2024-07-11 20:45:00,2024-07-11 20:45:00,0.0,0.0,Information,False,True,Information
2811477,6234297817509604666,2407112012,9,70,8098360,Bürstadt,68642,gernsheim,8002249.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
2811478,6234297817509604666,2407112012,10,70,8098360,Bürstadt,68642,groß-rohrheim,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message


In [15]:
"""european_train_stations = pd.read_csv("../train_stations_europe.csv")
european_train_stations.rename(columns={"uic":"IBNR", "longitude":"long", "latitude":"lat"},inplace=True)
merged_df = df_train_rides.merge(european_train_stations[['IBNR', 'lat', 'long']], on='IBNR', how='left', suffixes=('', '_european_train_stations'))

# Replace NaN lat/long in df_1 with values from df_2
df_train_rides['lat'] = df_train_rides['lat'].fillna(merged_df['lat_european_train_stations'])
df_train_rides['long'] = df_train_rides['long'].fillna(merged_df['long_european_train_stations'])

# Drop unnecessary columns added during merge
df_train_rides = df_train_rides.drop(columns=['lat_european_train_stations', 'long_european_train_stations'])
df_train_rides"""
european_train_stations = pd.read_csv("../train_stations_europe.csv")
# Rename columns in european_train_stations to match df_train_rides
european_train_stations.rename(columns={"uic": "IBNR", "longitude": "long", "latitude": "lat"}, inplace=True)
european_train_stations = european_train_stations[european_train_stations["country"]=="DE"]

# Filter rows where lat or long is NaN in df_train_rides
df_missing_lat_long = df_train_rides[df_train_rides['lat'].isna() | df_train_rides['long'].isna()]

# Split the missing data into 10 chunks
n_chunks = 10
chunk_size = len(df_missing_lat_long) // n_chunks + 1
chunks = np.array_split(df_missing_lat_long, n_chunks)

# Process each chunk sequentially
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}/{n_chunks}...")

    # Merge with european_train_stations for the current chunk
    merged_chunk = chunk.merge(
        european_train_stations[['IBNR', 'lat', 'long']],
        on='IBNR', how='left', suffixes=('', '_european_train_stations')
    )

    # Replace NaN lat/long in the original dataframe (df_train_rides) for the current chunk
    df_train_rides.loc[chunk.index, 'lat'] = df_train_rides.loc[chunk.index, 'lat'].fillna(merged_chunk['lat_european_train_stations'])
    df_train_rides.loc[chunk.index, 'long'] = df_train_rides.loc[chunk.index, 'long'].fillna(merged_chunk['long_european_train_stations'])

print("Processing complete.")
df_train_rides.to_csv("DBtrainrides_replaced_missing_coordinates.csv", index=False)

df_train_rides

Processing chunk 1/10...
Processing chunk 2/10...
Processing chunk 3/10...
Processing chunk 4/10...
Processing chunk 5/10...
Processing chunk 6/10...
Processing chunk 7/10...
Processing chunk 8/10...
Processing chunk 9/10...
Processing chunk 10/10...
Processing complete.


,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,city,zip,last_station,IBNR,long,...,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,canceled,info_present,transformed_info_message
0,-2065137557584893414,2407082237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,NaN,2024-07-08 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message
1,-2065137557584893414,2407092237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,NaN,2024-07-09 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message
2,-2065137557584893414,2407102237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,NaN,2024-07-10 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message
3,-2065137557584893414,2407112237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,NaN,2024-07-11 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message
4,-2065137557584893414,2407122237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,NaN,2024-07-12 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811475,6234297817509604666,2407112012,7,70,8098360,Bürstadt,68642,stockstadt(rhein),8005740.0,8.125738,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
2811476,6234297817509604666,2407112012,8,70,8098360,Bürstadt,68642,biebesheim,8000951.0,8.473978,...,2024-07-11 20:45:00,2024-07-11 20:45:00,2024-07-11 20:45:00,2024-07-11 20:45:00,0.0,0.0,Information,False,True,Information
2811477,6234297817509604666,2407112012,9,70,8098360,Bürstadt,68642,gernsheim,8002249.0,12.242452,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
2811478,6234297817509604666,2407112012,10,70,8098360,Bürstadt,68642,groß-rohrheim,NaN,12.078006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message


In [18]:
missing_long_lat = df_train_rides[(
                                  (df_train_rides['lat'].isna()) &
                                  (df_train_rides['long'].isna()))]

missing_long_lat

,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,city,zip,last_station,IBNR,long,...,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,canceled,info_present,transformed_info_message
57,1573967790757085557,2407072312,1,20,8000001,Aachen,52064,stolberg(rheinl)hbf gl.44,8000001.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
59,1573967790757085557,2407072312,3,20,8000001,Aachen,52064,alsdorf poststraße,8000510.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
61,1573967790757085557,2407072312,5,20,8000001,Aachen,52064,alsdorf-kellersberg,8000521.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
63,1573967790757085557,2407072312,7,20,8000001,Aachen,52064,alsdorf-busch,8000507.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
65,1573967790757085557,2407072312,9,20,8000001,Aachen,52064,herzogenrath-alt-merkstein,8002804.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811419,-8112742583161151520,2407081812,2,8,8098360,Bürstadt,68642,mannheim-käfertal,8003843.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
2811423,-8112742583161151520,2407091812,2,8,8098360,Bürstadt,68642,mannheim-käfertal,8003843.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
2811427,-8112742583161151520,2407101812,2,8,8098360,Bürstadt,68642,mannheim-käfertal,8003843.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message
2811431,-8112742583161151520,2407111812,2,8,8098360,Bürstadt,68642,mannheim-käfertal,8003843.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message


In [19]:
preprocessed_df = pd.read_csv("../station_coordinates_final_manually_updated.csv")

incomplete_df = df_train_rides[df_train_rides[["lat","long"]].isna().all(axis=1)]
df_sorted = incomplete_df.sort_values(by=['ID_Base', 'stop_number'])

# Shift the 'last_station' within each 'ID_Base' to get the next stop's 'last_station' in the current row
df_sorted['clear_station_name'] = df_sorted.groupby('ID_Base')['last_station'].shift(-1)
df_sorted

,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,city,zip,last_station,IBNR,long,...,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,canceled,info_present,transformed_info_message,clear_station_name
376610,-9222235769475763378,2407141220,7,72,8000338,Mindelheim,87719,rammingen(bay),8004927.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,NaN
1666621,-9221813992666955147,2407131244,9,8,8005504,Seefeld,82229,münchen leuchtenbergring,8004134.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,NaN
1895386,-9221103335547146071,2407081025,3,80,8006582,Würzburg,97072,muhr a see,8000534.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,muhr a see
1895401,-9221103335547146071,2407091025,3,80,8006582,Würzburg,97072,muhr a see,8000534.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,muhr a see
1895416,-9221103335547146071,2407101025,3,80,8006582,Würzburg,97072,muhr a see,8000534.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,muhr a see
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459296,9221055242504429929,2407101654,7,RB29,8000667,Montabaur,56410,steinefrenz,8005700.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,girod
459333,9221055242504429929,2407141654,8,RB29,8000667,Montabaur,56410,girod,8002280.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,NaN
440418,9221732241641229965,2407121114,4,RB20,8000596,Appenweier,77767,ibach,8003035.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,oppenau
440395,9221732241641229965,2407101114,5,RB20,8000596,Appenweier,77767,oppenau,8004679.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,zusenhofen


In [21]:
merged = pd.merge(
    df_sorted, preprocessed_df,
    on='clear_station_name',
    how='left',
    suffixes=('_original', '_new')
)

# Replace NaN values in 'long' and 'lat' using the values from the second DataFrame
merged['long'] = merged['long_original'].combine_first(merged['long_new'])
merged['lat'] = merged['lat_original'].combine_first(merged['lat_new'])

# Drop intermediate columns
result = merged.drop(columns=['long_original', 'long_new', 'lat_original', 'lat_new'])
result

,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,city,zip,last_station,IBNR,arrival_plan,...,departure_change,arrival_delay_m,departure_delay_m,info,canceled,info_present,transformed_info_message,clear_station_name,long,lat
0,-9222235769475763378,2407141220,7,72,8000338,Mindelheim,87719,rammingen(bay),8004927.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,NaN,116.96486,36.66644
1,-9221813992666955147,2407131244,9,8,8005504,Seefeld,82229,münchen leuchtenbergring,8004134.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,NaN,116.96486,36.66644
2,-9221103335547146071,2407081025,3,80,8006582,Würzburg,97072,muhr a see,8000534.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,muhr a see,10.72334,49.15241
3,-9221103335547146071,2407091025,3,80,8006582,Würzburg,97072,muhr a see,8000534.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,muhr a see,10.72334,49.15241
4,-9221103335547146071,2407101025,3,80,8006582,Würzburg,97072,muhr a see,8000534.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,muhr a see,10.72334,49.15241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142203,9221055242504429929,2407101654,7,RB29,8000667,Montabaur,56410,steinefrenz,8005700.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,girod,7.91100,50.45105
142204,9221055242504429929,2407141654,8,RB29,8000667,Montabaur,56410,girod,8002280.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,NaN,116.96486,36.66644
142205,9221732241641229965,2407121114,4,RB20,8000596,Appenweier,77767,ibach,8003035.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,oppenau,8.16498,48.47369
142206,9221732241641229965,2407101114,5,RB20,8000596,Appenweier,77767,oppenau,8004679.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,zusenhofen,8.01675,48.54725


In [23]:
missing_long_lat_after_api = result[(
        (result['lat'].isna()) &
        (result['long'].isna()))]

missing_long_lat_after_api

,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,city,zip,last_station,IBNR,arrival_plan,...,departure_change,arrival_delay_m,departure_delay_m,info,canceled,info_present,transformed_info_message,clear_station_name,long,lat
66,-9213894678850591087,2407091212,1,RB39,8000133,Grevenbroich,41515,bedburg(erft),8000133.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,bedburg(erft),NaN,NaN
67,-9213894678850591087,2407101212,1,RB39,8000133,Grevenbroich,41515,bedburg(erft),8000133.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,bedburg(erft),NaN,NaN
68,-9213894678850591087,2407111212,1,RB39,8000133,Grevenbroich,41515,bedburg(erft),8000133.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,bedburg(erft),NaN,NaN
69,-9213894678850591087,2407131212,1,RB39,8000133,Grevenbroich,41515,bedburg(erft),8000133.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,bedburg(erft),NaN,NaN
133,-9211731462094831414,2407142225,2,7,8002894,Höhenkirchen-Siegertsbrun,85635,pullach,8004899.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,großhesselohe isartalbf,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142071,9211785529441865905,2407091435,1,74,8000941,Bexbach,66450,illingen(saar),8000941.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,illingen(saar),NaN,NaN
142072,9211785529441865905,2407101435,1,74,8000941,Bexbach,66450,illingen(saar),8000941.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,illingen(saar),NaN,NaN
142073,9211785529441865905,2407111435,1,74,8000941,Bexbach,66450,illingen(saar),8000941.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,illingen(saar),NaN,NaN
142074,9211785529441865905,2407121435,1,74,8000941,Bexbach,66450,illingen(saar),8000941.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,illingen(saar),NaN,NaN


In [25]:
count_dict = missing_long_lat_after_api['clear_station_name'].value_counts().to_dict()
print(count_dict)

{'münchen hbf gl.27-36': 135, 'radebeul-zitzschewig': 86, 'flughafen ber (s-bahn)': 67, 'potsdam hbf (s)': 66, 'iserlohn': 62, 'ahrensfelde (s)': 61, 'berlin-spandau (s)': 58, 'bedburg(erft)': 53, 'erkner (s)': 51, 'bretten schulzentrum': 49, 'bremen hbf/hugo-schauinsland-platz, bremen': 45, 'hennigsdorf (s)': 41, 'bernau (s)': 40, 'küstrin-kietz': 38, 'wolfratshausen': 36, 'erding': 34, 'salzburg hbf': 34, 'halle-nietleben': 33, 'rückersbacher schlucht': 32, 'blankenfelde (s)': 32, 'greiz': 31, 'meißen triebischtal': 30, 'walporzheim': 30, 'hattingen(ruhr) mitte': 28, 'eppingen': 28, 'odenheim bf': 27, 'holländische straße, kassel': 27, 'gebra(hainleite)': 27, 'luxembourg': 26, 'dortmund-lütgendortmund': 25, 'burghaun(hünfeld)': 25, 'trebnitz(mark)': 25, 'esens(ostfriesl)': 24, 'oesede': 23, 'enschede': 23, 'altdorf(b nürnberg)': 23, 'wörth(rhein) badepark': 23, 'groß ammensleben': 23, 'kleve': 22, 'bremen-farge': 21, 'wissembourg': 21, 'kufstein': 21, 'xanten': 20, 'lemgo-lüttfeld': 

In [30]:
import os, requests
def get_coordinates_here(station_name, api_key):
    url = f"https://geocode.search.hereapi.com/v1/geocode?q={station_name}&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['items']:
            location = data['items'][0]['position']
            return location['lat'], location['lng']
    return None, None

# API Key for Here API
api_key = os.environ.get("HERE_API_KEY")

def update_lat_long(row, api_key):
    # Only update if lat or long is NaN
    if pd.isna(row['lat']) or pd.isna(row['long']):
        lat, long = get_coordinates_here(row['clear_station_name'], api_key)
        # Replace lat and long with the new coordinates
        row['lat'] = lat
        row['long'] = long
    return row

result = result.apply(update_lat_long, axis=1, api_key=api_key)
result.to_csv("test.csv",index=False)

In [32]:
result

,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,city,zip,last_station,IBNR,arrival_plan,...,departure_change,arrival_delay_m,departure_delay_m,info,canceled,info_present,transformed_info_message,clear_station_name,long,lat
0,-9222235769475763378,2407141220,7,72,8000338,Mindelheim,87719,rammingen(bay),8004927.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,NaN,116.96486,36.66644
1,-9221813992666955147,2407131244,9,8,8005504,Seefeld,82229,münchen leuchtenbergring,8004134.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,NaN,116.96486,36.66644
2,-9221103335547146071,2407081025,3,80,8006582,Würzburg,97072,muhr a see,8000534.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,muhr a see,10.72334,49.15241
3,-9221103335547146071,2407091025,3,80,8006582,Würzburg,97072,muhr a see,8000534.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,muhr a see,10.72334,49.15241
4,-9221103335547146071,2407101025,3,80,8006582,Würzburg,97072,muhr a see,8000534.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,muhr a see,10.72334,49.15241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142203,9221055242504429929,2407101654,7,RB29,8000667,Montabaur,56410,steinefrenz,8005700.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,girod,7.91100,50.45105
142204,9221055242504429929,2407141654,8,RB29,8000667,Montabaur,56410,girod,8002280.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,NaN,116.96486,36.66644
142205,9221732241641229965,2407121114,4,RB20,8000596,Appenweier,77767,ibach,8003035.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,oppenau,8.16498,48.47369
142206,9221732241641229965,2407101114,5,RB20,8000596,Appenweier,77767,oppenau,8004679.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,zusenhofen,8.01675,48.54725


In [37]:
missing_long_lat_after_second_cleaning = result[(
        (result['lat'].isna()) &
        (result['long'].isna()))]

missing_long_lat_after_second_cleaning = missing_long_lat_after_second_cleaning.apply(update_lat_long, axis=1, api_key=api_key)
missing_long_lat_after_second_cleaning

,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,city,zip,last_station,IBNR,arrival_plan,...,departure_change,arrival_delay_m,departure_delay_m,info,canceled,info_present,transformed_info_message,clear_station_name,long,lat
133,-9211731462094831414,2407142225,2,7,8002894,Höhenkirchen-Siegertsbrun,85635,pullach,8004899.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,großhesselohe isartalbf,None,None
2221,-8927383541084970752,2407072318,5,9,8013241,Waßmannsdorf,12529,berlin heerstraße,8089329.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,berlin messe süd (eichkamp),None,None
2442,-8903784507965862638,2407130944,4,RS4,8001155,Bremen,28199,brake(unterweser),8001120.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,kirchhammelwarden,None,None
2443,-8903784507965862638,2407120944,5,RS4,8001155,Bremen,28199,kirchhammelwarden,8003276.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,kirchhammelwarden,None,None
2621,-8870465185602086741,2407121608,10,56,8002400,Hanau,63457,alzenau burg,8077275.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,alzenau(unterfr),None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137204,8569444833499669882,2407090947,6,1,8010022,Bad Schandau,1814,radebeul-zitzschewig,8012701.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,radebeul-zitzschewig,None,None
138492,8721423322914546555,2407131310,2,L7,8010393,Zittau,2763,varnsdorf pivovar kocour,NaN,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,varnsdorf stare nadr,None,None
138999,8789838779519040058,2407122304,8,7,8004136,München,81669,pullach,8004899.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,großhesselohe isartalbf,None,None
139196,8806253552965513339,2407110637,6,19,8003990,Düren,52353,spich,8005629.0,NaN,...,NaN,NaN,NaN,NaN,False,False,No message,porz-wahn,None,None


In [44]:
combined_df = pd.concat([result, missing_long_lat_after_second_cleaning])

# Step 2: Drop duplicates based on the combination of 'ID_Base', 'ID_Timestamp', and 'stop_number'
final_result_df = combined_df.drop_duplicates(subset=['ID_Base', 'ID_Timestamp', 'stop_number'])

combined_df_all_rides = pd.concat([df_train_rides, final_result_df])

# Step 2: Drop duplicates based on the combination of 'ID_Base', 'ID_Timestamp', and 'stop_number'
final_result_df_all_rides = combined_df_all_rides.drop_duplicates(subset=['ID_Base', 'ID_Timestamp', 'stop_number'])

final_result_df_all_rides

,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,city,zip,last_station,IBNR,long,...,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,canceled,info_present,transformed_info_message,clear_station_name
0,-2065137557584893414,2407082237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,2024-07-08 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message,NaN
1,-2065137557584893414,2407092237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,2024-07-09 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message,NaN
2,-2065137557584893414,2407102237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,2024-07-10 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message,NaN
3,-2065137557584893414,2407112237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,2024-07-11 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message,NaN
4,-2065137557584893414,2407122237,1,29,8000001,Aachen,52064,NaN,8000001.0,6.091499,...,2024-07-12 22:37:00,NaN,NaN,0.0,0.0,NaN,True,False,No message,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811475,6234297817509604666,2407112012,7,70,8098360,Bürstadt,68642,stockstadt(rhein),8005740.0,8.125738,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,NaN
2811476,6234297817509604666,2407112012,8,70,8098360,Bürstadt,68642,biebesheim,8000951.0,8.473978,...,2024-07-11 20:45:00,2024-07-11 20:45:00,2024-07-11 20:45:00,0.0,0.0,Information,False,True,Information,NaN
2811477,6234297817509604666,2407112012,9,70,8098360,Bürstadt,68642,gernsheim,8002249.0,12.242452,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,NaN
2811478,6234297817509604666,2407112012,10,70,8098360,Bürstadt,68642,groß-rohrheim,NaN,12.078006,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,NaN


In [45]:
final_result_df_all_rides.to_csv("../DBtrainrides_restored_lat_long.csv",index=False)